<a href="https://colab.research.google.com/github/bbchen33/Data-Science/blob/master/Restaurant_data_PySpark_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Restaurant data from Kaggle
https://www.kaggle.com/shrutimehta/zomato-restaurants-data/download

In [3]:
!sudo apt install openjdk-8-jdk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jre x11-utils
Suggested packages:
  gvfs openjdk-8-demo openjdk-8-source visualvm icedtea-8-plugin mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jdk openjdk-8-jre x11-utils
0 upgraded, 13 newly installed, 0 to remove and 28 not upgraded.
Need to get 7,119 kB of archives.
After this operation, 20.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxxf86dga1 amd64 2:1.1.4-1 [13.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/m

In [4]:
!sudo update-alternatives --config java

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
* 0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
  2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


In [5]:
!java -version

openjdk version "1.8.0_222"
OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1ubuntu1~18.04.1-b10)
OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)


In [6]:
!pip install pyspark

     |████████████████████████████████| 215.7MB 68kB/s 
     |████████████████████████████████| 204kB 41.0MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=7bcb77dc12a76afecee717b431200c43ab0e053395c983f64b8872d03f4106a6
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
from google.colab import files

In [8]:
file = files.upload()

Saving zomato-restaurants-data.zip to zomato-restaurants-data.zip


In [9]:
!unzip zomato-restaurants-data.zip

Archive:  zomato-restaurants-data.zip
  inflating: Country-Code.xlsx       
  inflating: file1.json              
  inflating: file2.json              
  inflating: file3.json              
  inflating: file4.json              
  inflating: file5.json              
  inflating: zomato.csv              


In [10]:
!head zomato.csv

Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenue, Poblacion, Makati City","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Makati City",121.027535,14.565443,"French, Japanese, Desserts",1100,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Makati City",121.014101,14.553708,Japanese,1200,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Wa

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession \
  .builder \
  .appName('Restaurant') \
  .getOrCreate()

In [0]:
path = 'zomato.csv'
zomato_df = spark.read.csv(path, header = True)

In [13]:
zomato_df.describe().show()

+-------+--------------------+-----------------+------------------+------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+----------------+------------------+-------------------+-----------------+--------------------+------------------+------------------+-----------------+-----------------+------------------+
|summary|       Restaurant ID|  Restaurant Name|      Country Code|              City|             Address|            Locality|    Locality Verbose|        Longitude|            Latitude|            Cuisines|Average Cost for two|        Currency| Has Table booking|Has Online delivery|Is delivering now|Switch to order menu|       Price range|  Aggregate rating|     Rating color|      Rating text|             Votes|
+-------+--------------------+-----------------+------------------+------------------+--------------------+--------------------+--------------------+-------------

I'll write a simple for loop to replace all the white space in the column names to underscore so it's easier to manipulate by SQL.

In [14]:
for name in zomato_df.columns:
  zomato_df = zomato_df.withColumnRenamed(name, name.replace(' ', '_'))

print(zomato_df.columns)

['Restaurant_ID', 'Restaurant_Name', 'Country_Code', 'City', 'Address', 'Locality', 'Locality_Verbose', 'Longitude', 'Latitude', 'Cuisines', 'Average_Cost_for_two', 'Currency', 'Has_Table_booking', 'Has_Online_delivery', 'Is_delivering_now', 'Switch_to_order_menu', 'Price_range', 'Aggregate_rating', 'Rating_color', 'Rating_text', 'Votes']


In [15]:
zomato_df.take(1)

[Row(Restaurant_ID='6317637', Restaurant_Name='Le Petit Souffle', Country_Code='162', City='Makati City', Address='Third Floor, Century City Mall, Kalayaan Avenue, Poblacion, Makati City', Locality='Century City Mall, Poblacion, Makati City', Locality_Verbose='Century City Mall, Poblacion, Makati City, Makati City', Longitude='121.027535', Latitude='14.565443', Cuisines='French, Japanese, Desserts', Average_Cost_for_two='1100', Currency='Botswana Pula(P)', Has_Table_booking='Yes', Has_Online_delivery='No', Is_delivering_now='No', Switch_to_order_menu='No', Price_range='3', Aggregate_rating='4.8', Rating_color='Dark Green', Rating_text='Excellent', Votes='314')]

In [16]:
zomato_df.printSchema()

root
 |-- Restaurant_ID: string (nullable = true)
 |-- Restaurant_Name: string (nullable = true)
 |-- Country_Code: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Locality: string (nullable = true)
 |-- Locality_Verbose: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Cuisines: string (nullable = true)
 |-- Average_Cost_for_two: string (nullable = true)
 |-- Currency: string (nullable = true)
 |-- Has_Table_booking: string (nullable = true)
 |-- Has_Online_delivery: string (nullable = true)
 |-- Is_delivering_now: string (nullable = true)
 |-- Switch_to_order_menu: string (nullable = true)
 |-- Price_range: string (nullable = true)
 |-- Aggregate_rating: string (nullable = true)
 |-- Rating_color: string (nullable = true)
 |-- Rating_text: string (nullable = true)
 |-- Votes: string (nullable = true)



In [0]:
zomato_df.createOrReplaceTempView('zomato_df_table')

In [18]:
spark.sql('''
  SELECT * FROM zomato_df_table LIMIT 2
''').show()

+-------------+----------------+------------+-----------+--------------------+--------------------+--------------------+----------+---------+--------------------+--------------------+----------------+-----------------+-------------------+-----------------+--------------------+-----------+----------------+------------+-----------+-----+
|Restaurant_ID| Restaurant_Name|Country_Code|       City|             Address|            Locality|    Locality_Verbose| Longitude| Latitude|            Cuisines|Average_Cost_for_two|        Currency|Has_Table_booking|Has_Online_delivery|Is_delivering_now|Switch_to_order_menu|Price_range|Aggregate_rating|Rating_color|Rating_text|Votes|
+-------------+----------------+------------+-----------+--------------------+--------------------+--------------------+----------+---------+--------------------+--------------------+----------------+-----------------+-------------------+-----------------+--------------------+-----------+----------------+------------+-----

In [19]:
spark.sql('''
  SELECT distinct Rating_text FROM zomato_df_table
''').show()

+-----------+
|Rating_text|
+-----------+
|          7|
|  Excellent|
|          0|
|       null|
|    Average|
|       Good|
|  Very Good|
|  Not rated|
|       Poor|
+-----------+



In [21]:
spark.sql('''
  SELECT count(distinct Restaurant_Name)
  FROM zomato_df_table
''').show()

+-------------------------------+
|count(DISTINCT Restaurant_Name)|
+-------------------------------+
|                           7452|
+-------------------------------+



Which city has the most restaurants rated in this table?

In [29]:
spark.sql('''
  SELECT City, Count(distinct Restaurant_ID) as restaurant_count
  FROM zomato_df_table
  Group by City
  Order by restaurant_count desc
''').show()

+------------+----------------+
|        City|restaurant_count|
+------------+----------------+
|   New Delhi|            5473|
|     Gurgaon|            1118|
|       Noida|            1080|
|   Faridabad|             251|
|   Ghaziabad|              25|
|   Ahmedabad|              21|
|    Amritsar|              21|
|Bhubaneshwar|              21|
|     Lucknow|              21|
|    Guwahati|              21|
|   Bangalore|              20|
|    Auckland|              20|
|   Tampa Bay|              20|
|       Patna|              20|
|    Savannah|              20|
|       Kochi|              20|
|     Chennai|              20|
|   Edinburgh|              20|
|    Valdosta|              20|
|   Singapore|              20|
+------------+----------------+
only showing top 20 rows



What are the average ratings of restaurant in each city?

In [31]:
spark.sql('''
  SELECT City, AVG(Aggregate_rating) as average_rating
  FROM zomato_df_table
  Group by City
  Order by average_rating desc
''').show()

+----------------+------------------+
|            City|    average_rating|
+----------------+------------------+
|      Inner City|               4.9|
|     Quezon City|               4.8|
|     Makati City|              4.65|
|      Pasig City| 4.633333333333333|
|Mandaluyong City|             4.625|
|      Beechworth|               4.6|
|          London| 4.534999999999999|
|     Taguig City|             4.525|
|    Secunderabad|               4.5|
|   Tagaytay City|               4.5|
|         Lincoln|               4.5|
|         Orlando|4.4750000000000005|
|  Rest of Hawaii|              4.41|
|       Tampa Bay|              4.41|
|       Palm Cove|               4.4|
|         Tanunda|               4.4|
|       Bangalore| 4.375000000000001|
|           Dubai| 4.369999999999999|
|      Pasay City| 4.366666666666666|
|         Jakarta|           4.35625|
+----------------+------------------+
only showing top 20 rows



In [0]:
country_city_rating = spark.sql('''
  SELECT Country_Code, City, AVG(Aggregate_rating) OVER (Partition by City) as average_rating
  FROM zomato_df_table
  Order by average_rating desc
''')

In [0]:
country_city_rating.createOrReplaceTempView('country_city_rating_table')

In [68]:
spark.sql('''SELECT * FROM country_city_rating_table LIMIT 2''').show()

+------------+----------+--------------+
|Country_Code|      City|average_rating|
+------------+----------+--------------+
|         189|Inner City|           4.9|
|         189|Inner City|           4.9|
+------------+----------+--------------+



Which countries have the most cities with rating >= 4.5

In [71]:
spark.sql('''
  SELECT Country_code, count(distinct City) as city_count
  FROM country_city_rating_table
  Where average_rating >= 4.5
  Group by Country_Code
  Order by city_count desc
''').show()

+------------+----------+
|Country_code|city_count|
+------------+----------+
|         162|         6|
|           1|         1|
|          14|         1|
|         189|         1|
|         215|         1|
|         216|         1|
+------------+----------+



The restaurants with lowest cost but highest ratings in Singapore.

In [52]:
spark.sql('''
  SELECT City, Average_Cost_for_two, Aggregate_rating
  FROM zomato_df_table
  WHERE Average_Cost_for_two is not null 
  AND City = 'Singapore'
  Order by Average_Cost_for_two asc, Aggregate_rating desc
''').show()

+---------+--------------------+----------------+
|     City|Average_Cost_for_two|Aggregate_rating|
+---------+--------------------+----------------+
|Singapore|                 100|             4.1|
|Singapore|                  20|             4.2|
|Singapore|                 220|             3.8|
|Singapore|                 270|               4|
|Singapore|                  30|               3|
|Singapore|                 300|             3.9|
|Singapore|                 300|             3.4|
|Singapore|                 315|             3.9|
|Singapore|                  40|             3.9|
|Singapore|                  40|             3.7|
|Singapore|                 430|             3.8|
|Singapore|                  50|             3.8|
|Singapore|                  50|             3.2|
|Singapore|                 500|             3.8|
|Singapore|                  60|             3.2|
|Singapore|                  60|             3.1|
|Singapore|                  75|             3.2|
